In [1]:
import sys
import os
import copy
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import collections
from annoy import AnnoyIndex

In [2]:
debug=False
verbose=True
doBucketPlots=False
doIndividual=False
doProcessIndividual=False
doMerge=False
doBalance=True
doProcessMerged=True # for OutputBalanced do Min04, Min07, etc
#
list_minNbPositiveHit=[
    0,
    4,
    7,
    10,
]
list_mergeOption=[
    #[0,10],
    #[10,20],
    #[20,30],
    #[30,40],
    #[40,50],
    #[50,60],
    #[60,70],
    #[70,80],
    #[80,90],
    #[90,100],
    #[0,20],
    [0,30],
    #[0,40],
    #[0,50],
    #[0,60],
    #[0,70],
    #[0,80],
    #[0,90],
    #[0,100],
]
bucketSize=20

In [3]:
folderStem="/Volumes/Luiza_SSD"

#
inputFolderName=folderStem+"/ATLAS/TrackML/data/ttbar_mu200-generic"
#outputFolderName=folderStem+"/ATLAS/TrackML/output_general"
#outputFolderName=folderStem+"/ATLAS/TrackML/output_again"
outputFolderName=folderStem+"/ATLAS/TrackML/output_new"
# if output folder does not exist, create it
if not os.path.exists(outputFolderName):
    os.makedirs(outputFolderName)

In [4]:
def get_eventNumber_from_fileName(fileName):
    eventNumber=fileName.replace("event","").replace("-hits.csv","")
    return eventNumber

In [5]:
# calculate event numbers from my folder
# lista goala in care sa pun numerele evenimentelor
list_eventNumber=[]
# The sorted() function returns a sorted list of the specified iterable object 
# Strings are sorted alphabetically, and numbers are sorted numerically
# os operating system 
# hits ca sa nu am de doura ori evenimentul in lista
for fileName in sorted(os.listdir(inputFolderName)):
    if fileName.endswith("-hits.csv"):
        #print(fileName)
        eventNumber=get_eventNumber_from_fileName(fileName)
        #print(eventNumber)
        list_eventNumber.append(eventNumber)
# done for loop
#list_eventNumber=list_eventNumber[0:20] # keep only first 10 events
#list_eventNumber=["000000007"] # keep only one event for test
print("All events available in my folder. list_eventNumber", list_eventNumber)

All events available in my folder. list_eventNumber ['000000000', '000000001', '000000002', '000000003', '000000004', '000000005', '000000006', '000000007', '000000008', '000000009', '000000010', '000000011', '000000012', '000000013', '000000014', '000000015', '000000016', '000000017', '000000018', '000000019', '000000020', '000000021', '000000022', '000000023', '000000024', '000000025', '000000026', '000000027', '000000028', '000000029', '000000030', '000000031', '000000032', '000000033', '000000034', '000000035', '000000036', '000000037', '000000038', '000000039', '000000040', '000000041', '000000042', '000000043', '000000044', '000000045', '000000046', '000000047', '000000048', '000000049', '000000050', '000000051', '000000052', '000000053', '000000054', '000000055', '000000056', '000000057', '000000058', '000000059', '000000060', '000000061', '000000062', '000000063', '000000064', '000000065', '000000066', '000000067', '000000068', '000000069', '000000070', '000000071', '000000072'

In [6]:
def p(name,nparray):
    print("Start",name)
    print(nparray)
    print("End",name,"shape",nparray.shape,"dtype",nparray.dtype,"type",type(nparray))
# done function

In [7]:
def calculate_eta(px,py,pz,m):
    p=np.sqrt(px*px+py*py+pz*pz)
    theta=np.arccos(pz/p)
    eta=-np.log(np.tan(theta/2))
    return eta
# done function

# example
eta=calculate_eta(3,4,100000,0)
eta

10.59663469141339

In [8]:
# this runs only once per event
def buildAnnoyIndex(nparray_position,metric="angular",ntrees=10,debug=False): 
    
    numberDimension=nparray_position.shape[1] # 3 (x,y,z)
    if debug:
        print("numberDimension",numberDimension,"metric",metric)
    index=AnnoyIndex(numberDimension,metric)
    if debug:
        print("type(index)",type(index))
        print("enumerate data")
    # add each hit to the index
    for i,position in enumerate(nparray_position):
        if debug:
            print("i",i,"position",position)
        index.add_item(i,position)
    # done for loop over hits
    # build the index with 10 trees
    index.build(ntrees) # 10 trees
    return index
# done function

In [9]:
def get_input_output_from_df_hits(df_hits,index):
    list_nparray_input=[]
    list_nparray_output=[]
    list_nparray_eta=[]
    list_nparray_volumeID=[]
    list_nparray_layerID=[]
    list_nparray_queryIndex=[]
    list_nparray_nbPositiveHit=[]
    
    nparray_volume_id=df_hits["volume_id"].values
    nparray_layer_id=df_hits["layer_id"].values 

    counterBucket=0
    
    #my_title="FirstLayer"
    my_title="All"
    for i in range(len(df_hits)):
        if (
            True # for each hit create a bucket
            #(nparray_volume_id[i]==8 and nparray_layer_id[i]==2) 
            #or (nparray_volume_id[i]==7 and nparray_layer_id[i]==2)
            #or (nparray_volume_id[i]==9 and nparray_layer_id[i]==14)
            #or (nparray_volume_id[i]==16 and nparray_layer_id[i]==2)
            #or (nparray_volume_id[i]==12 and nparray_layer_id[i]==2)
            #or (nparray_volume_id[i]==18 and nparray_layer_id[i]==12)
            #or (nparray_volume_id[i]==14 and nparray_layer_id[i]==12)
            #or (nparray_volume_id[i]==17 and nparray_layer_id[i]==4)
            )==False:
            continue

        counterBucket+=1
        
        #if (counterBucket<1)==False:
        #    continue
        
        #if (counterBucket%10000==1)==False:
        #    continue
        
        if debug or counterBucket%20000==1:
            print("i",i,"counterBucket",counterBucket)
            
        # using annoy to find the 20 nearest neighboring hits by angle to this hit
        list_index=index.get_nns_by_item(i,bucketSize)
        if debug:
            print("list_index",list_index)
            
        # create bucket
        df_bucket=df_hits.iloc[list_index]
        df_query=df_hits.iloc[i]
        
        if debug:
            print("df_bucket",df_bucket)
            print("df_query",df_query)
            print("df_query_hit_id",df_query["hit_id"])
        
        # find the position (index) in the bucket of the query point and store it for later user
        query_hitID=df_query["hit_id"].astype(np.uint64)
        if debug:
            print("query_hitID",query_hitID) 
        # for loop over hits in bucket and remember at which position (index) inside the bucket we have the query point
        nparray_hitID=df_bucket["hit_id"].values.astype(np.uint64)
        for j in range(bucketSize):
            current_hitID=nparray_hitID[j]
            if debug:
                print("current_hitID",current_hitID)
            if current_hitID==query_hitID:
                queryIndex=j
                break
        # done for loop over hits in bucket
        if debug:
            print("queryIndex",queryIndex)
        # create numpy array for queryIndex
        list_queryIndex=[queryIndex]
        nparray_queryIndex=np.array(list_queryIndex).astype(np.uint8)
        
        # create NN input for this one bucket
        # convert from default in pandas from np.float64 to np.float32, which is enough for the 4-5 digits it has
        nparray_input=df_bucket[["x","y","z"]].values.astype(np.float32).flatten()
        if debug:
            p("nparray_input",nparray_input)
            
        # as for input, for volumeID
        nparray_volumeID=df_bucket["volume_id"].values.astype(np.uint8)
        
        # as for input, for layerID
        nparray_layerID=df_bucket["layer_id"].values.astype(np.uint8)
            
        # create NN output for this one bucket
        # identify particle with the largest number of hits in this bucket
        nparray_particleID=df_bucket["particle_id"].values

        if debug:
            p("nparray_particleID",nparray_particleID)
        dict_particleID_counterParticleID={}
        for particleID in nparray_particleID:
            if particleID not in dict_particleID_counterParticleID:
                dict_particleID_counterParticleID[particleID]=1
            else:
                dict_particleID_counterParticleID[particleID]+=1
        if debug:
            print("dict_particleID_counterParticleID",dict_particleID_counterParticleID)
          
        # find the maximum value of the counters
        particleIDWithMaxHits=0
        counterParticleIDWithMaxHits=0
        for particleID in dict_particleID_counterParticleID:
            counterParticleID=dict_particleID_counterParticleID[particleID]
            if counterParticleID>counterParticleIDWithMaxHits:
                counterParticleIDWithMaxHits=counterParticleID
                particleIDWithMaxHits=particleID
        if debug:
            print("counterBucket",counterBucket,"particleIDWithMaxHits",particleIDWithMaxHits,"counterParticleIDWithMaxHits",counterParticleIDWithMaxHits)           
        
        # store the number of positive hits in this bucket, as it will be used a lot later
        nparray_nbPositiveHit=np.array([counterParticleIDWithMaxHits]).astype(np.uint8)
        
        # calculate eta
        nparray_tp=df_bucket[["tpx","tpy","tpz"]].values.astype(np.float32)
        if debug:
            p("nparray_tp",nparray_tp)
        
        # calculate eta for each hit and store it
        if debug:
            print("Start eta")
            print("particleIDWithMaxHits",particleIDWithMaxHits)
            
        # for loop over hits in a bucket
        # create list of eta for each hit in the bucket
        list_eta=[]
        for j,particleID in enumerate(nparray_particleID):
            #print("j",j,"particleID",particleID)
            list_tp=nparray_tp[j]
            # print("list_tp",list_tp)
            tpx=list_tp[0]
            tpy=list_tp[1]
            tpz=list_tp[2]
            # let's assume mass of particle is zero (good approximationf for pions, kaons forming tracks)
            m=0.0
            #print("j",j,"tpx=%.4f, tpy=%.4f, tpz=%.4f"%(tpx,tpy,tpz))
            # 
            eta=calculate_eta(tpx,tpy,tpz,m)
            if debug:
                print("j",j,"particleID",particleID,"tpx=%.10f, tpy=%.10f, tpz=%.10f, eta=%.10f"%(tpx,tpy,tpz,eta))
            list_eta.append(eta)
        # done for loop over hits in bucket
        # by printing out I confirmed that all hits belonging to the particle have exactly the same tpx,tpy,tpz
        # list_eta has bucketSize elements, so now make it a numpy array and convert it to type float 32
        nparray_eta=np.array(list_eta).astype(np.float32)
        if debug:
            p("nparray_eta",nparray_eta)
        
        # create nparray_output
        list_output=[]
        
        # loop over every hit in the bucket
        for particleID in nparray_particleID:      
            if particleID==particleIDWithMaxHits:
                list_output.append(1)
            else:
                list_output.append(-1)
            # done if
        # done for loop for each hit in the bucket
        if debug:        
            print("list_output",list_output)
        # create nparray for output from list and convert from the default int64 to int8
        nparray_output=np.array(list_output).astype(np.int8)
        if debug:
            print("nparray_output",nparray_output)
            
        if doBucketPlots and counterBucket%1000==0:
            print("i",i,"counterBucket",counterBucket)
            
            outputFileNameStem=outputFolderName+"/plot_vis_e_"+eventNumber+"_i_"+str(i)+"_c_"+str(counterBucket)+"_"+my_title+"_"
            if True:
                # plot x vs y
                plt.plot(df_bucket.x,df_bucket.y,"o",color="red")
                plt.plot(df_query.x,df_query.y,"D",color="blue")
                plt.xlabel("x [mm]")
                plt.ylabel("y [mm]")
                plt.xlim(left=-1050,right=1050)
                plt.ylim(bottom=-1050,top=1050)
                #plt.plot(0,0,"r+")
                plt.title(my_title+". y vs x. Blue=query. eta=%.2f. i=%.0f"%(eta,i))
                # plt.show()
                plotBucketFileNameStem=outputFileNameStem+"y_vs_x"
                plt.savefig(plotBucketFileNameStem+".png")
                #plt.savefig(plotBucketFileNameStem+".pdf")
                plt.close()
            # done if
            if True:
                # plot z vs r
                df_bucket_r=np.sqrt(df_bucket.x**2+df_bucket.y**2)
                df_query_r=np.sqrt(df_query.x**2+df_query.y**2)
                plt.plot(df_bucket.z,df_bucket_r,"o",color="red")
                plt.plot(df_query.z,df_query_r,"D",color="blue")
                plt.xlabel("z [mm]")
                plt.ylabel("radius [mm]")
                plt.xlim(left=-3050,right=3050)
                plt.ylim(bottom=-1050,top=1050)
                #plt.plot(0,0,"r+")
                plt.title(my_title+". r vs z. Blue=query. eta=%.2f. i=%.0f"%(eta,i))
                # plt.show()
                plotBucketFileNameStem=outputFileNameStem+"r_vs_z"
                plt.savefig(plotBucketFileNameStem+".png")
                #plt.savefig(plotBucketFileNameStem+".pdf")
                plt.close()
            # done if
        # done if
            
        # add for this current bucket to the list for the entire event (which has several buckets)
        list_nparray_input.append(nparray_input)
        list_nparray_output.append(nparray_output)
        list_nparray_eta.append(nparray_eta)
        list_nparray_volumeID.append(nparray_volumeID)
        list_nparray_layerID.append(nparray_layerID)
        list_nparray_queryIndex.append(nparray_queryIndex)
        list_nparray_npPositiveHit.append(nparray_nbPositiveHit)
    # done for loop over hits in the event, so that for each hit we make a bucket
    
    # transform list of numpy array into numpy array of dimension 2 
    # rows are buckets, columns are intput and output values
    nparray_input_all=np.array(list_nparray_input)
    nparray_output_all=np.array(list_nparray_output)
    nparray_eta_all=np.array(list_nparray_eta)
    nparray_volumeID_all=np.array(list_nparray_volumeID)
    nparray_layerID_all=np.array(list_nparray_layerID)
    nparray_queryIndex_all=np.array(list_nparray_queryIndex)
    nparray_nbPositiveHit_all=np.array(list_nparray_nbPositiveHit)
    if debug:
        p("nparray_input_all",nparray_input_all)
        p("nparray_output_all",nparray_output_all)
        p("nparray_eta_all",nparray_eta_all)
        p("nparray_volumeID_all",nparray_volumeID_all)
        p("nparray_layerID_all",nparray_layerID_all)
        p("nparray_queryIndex_all",nparray_queryIndex_all)
        p("nparray_nbPositiveHit_all",nparray_nbPositiveHit_all)
    
    return nparray_input_all, nparray_output_all, nparray_eta_all, nparray_volumeID_all, nparray_layerID_all, nparray_queryIndex_all, nparray_nbPositiveHit_all
# done function      

In [10]:
def do_run_one_event_at_a_time(list_eventNumber):
    # for loop over eventNumber from list_eventNumber and write individual event to files
    for i,eventNumber in enumerate(list_eventNumber):
        #if i>0:
        #    continue
    
        print()
        print()
        print("******************************")
        print("i",i,"eventNumber",eventNumber)
        print("******************************")
    
        inputFileName_hits_recon=inputFolderName+"/event"+eventNumber+"-hits.csv"
        inputFileName_hits_truth=inputFolderName+"/event"+eventNumber+"-truth.csv"
    
        if debug or verbose:
            print("Read csv files ad df and merge them")
        df_hits_recon=pd.read_csv(inputFileName_hits_recon)
        df_hits_truth=pd.read_csv(inputFileName_hits_truth)
        # hit_id appears in both, erase from one so that when we concatenate below it appears only once
        del df_hits_truth["hit_id"]
        df_hits=pd.concat([df_hits_recon,df_hits_truth],axis=1,sort=False)
    
        # build annoy index
        if debug or verbose:
            print("Start do Annoy Index")
        nparray_position=df_hits[["x","y","z"]].values
        if debug:
            p("nparray_position",nparray_position)

        index=buildAnnoyIndex(nparray_position,metric="angular",ntrees=10,debug=False)
    
        # for loop peste hituri peste hiturile preselectate volume_id=8, layer_id=2
        if debug or verbose:
            print("From df_hits produce nparray_input_all,nparray_output_all")
        nparray_input_all,nparray_output_all,nparray_eta_all,nparray_volumeID_all,nparray_layerID_all,nparray_queryIndex_all,nparray_nbPositiveHit_all=get_input_output_from_df_hits(df_hits,index)
    
        if debug:
            p("nparray_input_all",nparray_input_all)
            p("nparray_output_all",nparray_output_all)
            p("nparray_eta_all",nparray_eta_all)
            p("nparray_volumeID_all",nparray_volumeID_all)
            p("nparray_layerID_all",nparray_layerID_all)
            p("nparray_queryIndex_all",nparray_queryIndex_all)
            p("nparray_nbPositiveHit_all",nparray_nbPositiveHit_all)
    
        # reshape only input by adding one extra dimension needed by tensorflow, in practice it puts all into one extra bracket
        nparray_input_all=nparray_input_all.reshape(nparray_input_all.shape[0],nparray_input_all.shape[1],1)
        if debug or verbose:
            p("nparray_input_all after reshape",nparray_input_all)

        fileNameNNInputAll=outputFolderName+"/NN_2_data_Input_"+eventNumber+".npy"
        fileNameNNOutputAll=outputFolderName+"/NN_2_data_Output_"+eventNumber+".npy"
        fileNameNNEtaAll=outputFolderName+"/NN_2_data_Eta_"+eventNumber+".npy"
        fileNameNNVolumeIDAll=outputFolderName+"/NN_2_data_VolumeID_"+eventNumber+".npy"
        fileNameNNLayerIDAll=outputFolderName+"/NN_2_data_LayerID_"+eventNumber+".npy"
        fileNameNNQueryIndexAll=outputFolderName+"/NN_2_data_QueryIndex_"+eventNumber+".npy"
        fileNameNNNbPositiveHitAll=outputFolderName+"/NN_2_data_NbPositiveHit_"+eventNumber+".npy"
            
        np.save(fileNameNNInputAll,nparray_input_all)
        np.save(fileNameNNOutputAll,nparray_output_all)
        np.save(fileNameNNEtaAll,nparray_eta_all)
        np.save(fileNameNNVolumeIDAll,nparray_volumeID_all)
        np.save(fileNameNNLayerIDAll,nparray_layerID_all)
        np.save(fileNameNNQueryIndexAll,nparray_queryIndex_all)
        np.save(fileNameNNNbPositiveHitAll,nparray_nbPositiveHit_all)

    # done for loop over events
# done function

In [11]:
if doIndividual:
    do_run_one_event_at_a_time(list_eventNumber)

In [12]:
def do_create_output_based_on_min_nb_of_positive_hits(list_eventNumber):
    nparray_output_negative_one_bucket=np.array([-1 for i in range(bucketSize)]).astype(np.int8)
    # for loop over eventNumber from list_eventNumber
    for i,eventNumber in enumerate(list_eventNumber):
        if debug or verbose:
            print("eventNumber",eventNumber)
        # open the files needed for this event: Output and NbPositiveHit
        fileNameNNOutputAll=outputFolderName+"/NN_2_data_Output_"+eventNumber+".npy"
        nparray_output_all=np.load(fileNameNNOutputAll)
        fileNameNNNbPositiveHitAll=outputFolderName+"/NN_2_data_NbPositiveHit_"+eventNumber+".npy"
        nparray_nbPositiveHit_all=np.load(fileNameNNNbPositiveHitAll)
        # 
        dict_minNbPositiveHit_nparray_output_all={}
        # for loop over each val of min number of positive fits
        for minNbPositiveHit in list_minNbPositiveHit:
            if debug:
                print("i",i,"eventNumber",eventNumber,"minNbPositiveHit",minNbPositiveHit)
            # copy the current output in a completely new version with deep copy
            # as we will want to replace only the values that need changed for less than needed positive hits
            dict_minNbPositiveHit_nparray_output_all[minNbPositiveHit]=copy.deepcopy(nparray_output_all)
        # done for loop over minNbPositiveHit
        #
        # loop over each row (bucket) in the output
        for j in range(len(nparray_output_all)):
            bucket=nparray_output_all[j]
            if debug:
                p("j=%.0f, bucket"%j,bucket)
            nbPositiveHit=nparray_nbPositiveHit_all[j]
            if debug:
                print("j=%.0f, nbPositiveHit=%.0f"%(j,nbPositiveHit))
            # to speed up, change only wheen needed
            for minNbPositiveHit in list_minNbPositiveHit:
                if nbPositiveHit<minNbPositiveHit:
                    # replace this one line with the one with all negative
                    dict_minNbPositiveHit_nparray_output_all[minNbPositiveHit][j]=nparray_output_negative_one_bucket
                if debug:
                    print("j=%.0f, minNbPositiveHit=%.0f, bucket"%(j,minNbPositiveHit),dict_minNbPositiveHit_nparray_output_all[minNbPositiveHit][j])
            # done for loop over minNbPositiveHit  
        # done for loop over buckets
        # for this event, write the new processed output to file
        for minNbPositiveHit in list_minNbPositiveHit:
            minString="Min"
            if minNbPositiveHit<9:
                minString+="0"
            minString+=str(minNbPositiveHit)
            fileNameNNOutputAll=outputFolderName+"/NN_2_data_Output"+minString+"_"+eventNumber+".npy"
            np.save(fileNameNNOutputAll,dict_minNbPositiveHit_nparray_output_all[minNbPositiveHit])
        # done for loop over minNbPositiveHit  
    # done for loop over events
# done function
if debug or verbose:
    print("Done do_create_output_based_on_min_nb_of_positive_hits(list_eventNumber")

Done do_create_output_based_on_min_nb_of_positive_hits(list_eventNumber


In [13]:
if doProcessIndividual:
    do_create_output_based_on_min_nb_of_positive_hits(list_eventNumber)

In [14]:
def do_merge_events(mergeOption):
    # create output folder name for the merge, for the current number of events to merge
    eventNumberMin=mergeOption[0] # included
    eventNumberMax=mergeOption[1] # excluded
    #
    outputFolderNameMerge=outputFolderName+"_ev_"
    #
    if eventNumberMin<10:
        outputFolderNameMerge+="00"
    elif eventNumberMin<100:
        outputFolderNameMerge+="0"
    outputFolderNameMerge+=str(eventNumberMin)
    #
    outputFolderNameMerge+="_"
    #
    if eventNumberMax<10:
        outputFolderNameMerge+="00"
    elif eventNumberMax<100:
        outputFolderNameMerge+="0"
    outputFolderNameMerge+=str(eventNumberMax)
    #
    if not os.path.exists(outputFolderNameMerge):
        os.makedirs(outputFolderNameMerge)
    
    # for loop over events and merge them in train or test and save in a different folder
    counter_event_Train=0
    counter_event_Test=0
    if debug or verbose:
        print("Start loop over all events and merge them in either train or test")
    
    # list of variables to merge
    list_var=[
        "Input",
        "Output",
        "VolumeID",
        "LayerID",
        "Eta",
        "QueryIndex",
        "NbPositiveHit",
    ]
    # add to list_var also the OutputMin04, OutputMin07, etc
    for minNbPositiveHit in list_minNbPositiveHit:
        minString="OutputMin"
        if minNbPositiveHit<9:
            minString+="0"
        minString+=str(minNbPositiveHit)
        list_var.append(minString)
    # done for loop over minNbPositiveHit
    if debug:
        print("list_var",list_var)
    
    #
    dict_nameMerged_nparray={}
    
    # loop over the events, but careful, not all, but only from the first nrEventsToMerge
    for i,eventNumber in enumerate(list_eventNumber[eventNumberMin:eventNumberMax]):
        print("i",i,"eventNumber",eventNumber)
        
        # for loop over all variables
        dict_var_nparray={}
        for var in list_var:
            # read the stored file
            fileName=outputFolderName+"/NN_2_data_"+var+"_"+eventNumber+".npy"
            dict_var_nparray[var]=np.load(fileName)
        # done for loop over var
        
        # calculate the rest of the event index to division by 100
        # assuming we only merge multiples of ten, for each 10, we put first 7 in Train and last 3 in Test
        rest=i%10

        if rest<7: 
            # put this event into Train, with rest=i%10=0,1,2,3,4,5,6
            counter_event_Train+=1
            if counter_event_Train==1:
                # it is the first event of type Train, so we simply deep copy it to the output
                for var in list_var:
                    dict_nameMerged_nparray["Train"+var]=copy.deepcopy(dict_var_nparray[var])
            else:
                # it is not the first, so we concatentane to the new output
                for var in list_var:
                    dict_nameMerged_nparray["Train"+var]=np.concatenate((dict_nameMerged_nparray["Train"+var],dict_var_nparray[var]),axis=0,out=None)
            # done if
        else:
            # put this event into Test, with rest=i%10=7,8,9
            counter_event_Test+=1
            if counter_event_Test==1:
                # it is the first event of type Test, so we simply deep copy it to the output
                for var in list_var:
                    dict_nameMerged_nparray["Test"+var]=copy.deepcopy(dict_var_nparray[var])
            else:
                # it is not the first, so we concatentane to the new output
                for var in list_var:
                    dict_nameMerged_nparray["Test"+var]=np.concatenate((dict_nameMerged_nparray["Test"+var],dict_var_nparray[var]),axis=0,out=None)
            # done if
        # done if  
    # done for loop over eventNumber

    # write file to the output
    eventNumber="all"
    for TrainOrTest in "Train,Test".split(","):
        for var in list_var:
            fileName=outputFolderNameMerge+"/NN_2_data_"+var+"_"+TrainOrTest+"_"+eventNumber+".npy"
            np.save(fileName,dict_nameMerged_nparray[TrainOrTest+var])
            if debug:
                p("nparray_"+TrainOrTest+"_"+var+"_all_events",dict_nameMerged_nparray[TrainOrTest+var])
        # done for loop over var
    # done for loop over TrainOrTest
    
# done function

In [15]:
if doMerge:
    for mergeOption in list_mergeOption:
        print()
        print()
        print("******************************")
        print("mergeOption",mergeOption)
        print("******************************")
        do_merge_events(mergeOption)
        if debug:
            print("Done mergeOption",mergeOption)
    # done for loop over mergeOption
# done if

In [16]:
def do_balance(TrainOrTest,nparray_input,nparray_output,nparray_nbPositiveHit,nparray_eta,nparray_layerID,nparray_volumeID,nparray_queryIndex,outputFolderNameMerge,doVerifyForHit=False,debug=False,verbose=False):
    if debug or verbose:
        print("Start get_nparray_weight for TrainOrTest=",TrainOrTest)
 
    if verbose:
        p("nparray_input",nparray_input)
        p("nparray_output",nparray_output)
        p("nparray_nbPositiveHit",nparray_nbPositiveHit)
        p("nparray_eta",nparray_eta)
        p("nparray_layerID",nparray_layerID)
        p("nparray_volumeID",nparray_volumeID)
        p("nparray_queryIndex",nparray_queryIndex)

    # total number of buckets
    nbBucket=nparray_nbPositiveHit.shape[0]
    nrHitTotal=nbBucket*bucketSize
    if debug or verbose:
        print()
        print()
        print(TrainOrTest+":")
        print("nbBucket",nbBucket,"bucketSize",bucketSize,"nrHitTotal",nrHitTotal)
        
        
    # dictionary between number of positive hits in a bucket and the number of buckets with this number of positive hits
    dict_nbPositiveHit_counterBucket_unbalanced={}
    dict_nbPositiveHit_counterBucket_balancedDesired={}
    dict_nbPositiveHit_counterBucket_balancedCounter={}
    dict_nbPositiveHit_counterBucket_balancedObtained={}
    # create explicit the list of posssible values for nbPositiveHit (0, 1, 2, ..., 19, 20)
    list_nbPositiveHit=[i for i in range(0,21)]
    for nbPositiveHit in list_nbPositiveHit:
        dict_nbPositiveHit_counterBucket_unbalanced[nbPositiveHit]=0
        dict_nbPositiveHit_counterBucket_balancedDesired[nbPositiveHit]=0
        dict_nbPositiveHit_counterBucket_balancedCounter[nbPositiveHit]=0
        dict_nbPositiveHit_counterBucket_balancedObtained[nbPositiveHit]=0
    
    # now fill the dictionary of the counters depending on nbPositiveHit
    # loop over buckets
    for i in range(nbBucket):
        nbPositiveHit=nparray_nbPositiveHit[i][0] # one column only, so take that element index [0]
        if debug:
            print("nbPositiveHit",nbPositiveHit)
        dict_nbPositiveHit_counterBucket_unbalanced[nbPositiveHit]+=1

    # now dictionary is filled and we can print the values
    if debug or verbose:
        print()
        print("Original unbalanced. "+TrainOrTest+":")
        for nbPositiveHit in list_nbPositiveHit:
            counterBucket=dict_nbPositiveHit_counterBucket_unbalanced[nbPositiveHit]
            percentBucket=100*counterBucket/nbBucket
            print("nbPositiveHit=%.0f"%nbPositiveHit,"counterBucket=%.0f"%counterBucket,"percentBucket=%.1f"%percentBucket)
            
    # plot legend position
    legend_position="upper right"
            
    # make plots
    # we want bins from 0 to 21, as bin 20 is between 20 and 21
    bins=[i-0.5 for i in range(0,22)]
    print("bins",bins)
    # plot not normalized
    plt.hist(nparray_nbPositiveHit,bins=bins,histtype="step",color="red",label="Unbalanced",)
    plt.title("Histogram not normalized for "+TrainOrTest)
    plt.xlabel("Number of positive hits per bucket")
    plt.ylabel("Number of buckets")
    plt.legend(loc=legend_position)
    plt.tight_layout()
    #plt.show()
    plt.savefig(outputFolderNameMerge+"/plot_histo_1_NbBucket_vs_NbPositiveHit_"+TrainOrTest+".png")
    plt.savefig(outputFolderNameMerge+"/plot_histo_1_NbBucket_vs_NbPositiveHit_"+TrainOrTest+".pdf")
    plt.close()
    # plot normalized
    plt.hist(nparray_nbPositiveHit,bins=bins,density=True,histtype="step",color="red",label="Unbalanced")
    plt.title("Histogram normalized for "+TrainOrTest)
    plt.xlabel("Number of positive hits per bucket")
    plt.ylabel("Number of buckets")
    plt.legend(loc=legend_position)
    plt.tight_layout()
    #plt.show()
    plt.savefig(outputFolderNameMerge+"/plot_histo_1_NbBucket_vs_NbPositiveHit_normalized_"+TrainOrTest+".png")
    plt.savefig(outputFolderNameMerge+"/plot_histo_1_NbBucket_vs_NbPositiveHit_normalized_"+TrainOrTest+".pdf")
    plt.close()
    
    # we want to balance out around 10 by removing not needed buckets, to balance 9 with 11, 8 with 12, etc
    
    # first calculate the desired number of buckets in each category, so that it is balanced around 10
    for i in range(bucketSize//2+1):
        #print("i",i)
        val_left=dict_nbPositiveHit_counterBucket_unbalanced[i]
        #print("val_left",val_left)
        val_right=dict_nbPositiveHit_counterBucket_unbalanced[bucketSize-i]
        #print("val_right",val_right)
        if val_left<=val_right:
            val_min=val_left
        else:
            val_min=val_right
        dict_nbPositiveHit_counterBucket_balancedDesired[i]=val_min
        dict_nbPositiveHit_counterBucket_balancedDesired[bucketSize-i]=val_min
    # done for loop over i 

    # now dictionary is filled and we can print the values
    if debug or verbose:
        print()
        print("BalancedDesired. "+TrainOrTest+":")
        for nbPositiveHit in list_nbPositiveHit:
            counterBucket=dict_nbPositiveHit_counterBucket_balancedDesired[nbPositiveHit]
            percentBucket=100*counterBucket/nbBucket
            print("nbPositiveHit=%.0f"%nbPositiveHit,"counterBucket=%.0f"%counterBucket,"percentBucket=%.1f"%percentBucket)
    

    # we want a nparray_inputC that will have a subset of nparray_input, same for nparray_output
    list_inputBalanced=[]
    list_outputBalanced=[]
    list_nbPositiveHitBalanced=[]
    list_etaBalanced=[]
    list_layerIDBalanced=[]
    list_volumeIDBalanced=[]
    list_queryIndexBalanced=[]
    # loop over buckets
    
    print()
    for i in range(nbBucket):
        # first decide if I need to keep this bucket or not
        # first find the nbPositiveHit of this bucket
        nbPositiveHit=nparray_nbPositiveHit[i][0]
        # find how many buckets I am allowed to keep in this category
        nrBucketDesired=dict_nbPositiveHit_counterBucket_balancedDesired[nbPositiveHit]
        #print("j",j,"nbPositiveHit",nbPositiveHit,"nrBucketDesired",nrBucketDesired)
        # find if this bucket is allowed, meaning if the counter of this bucketg is smaller than allowed
        # so I need to find the current counter
        dict_nbPositiveHit_counterBucket_balancedCounter[nbPositiveHit]+=1
        # now counter is increased and I can make the check, so first bucket has counter 1 (not start from 0)
        # if current current counter smaller or equal with the allowed, you write the current bucket
        # if not, you skip it
        #print("nbPositiveHit",nbPositiveHit,"nrBucketDesired",nrBucketDesired,"counter",dict_nbPositiveHit_counterBucket_balancedObtained[nbPositiveHit])
        if dict_nbPositiveHit_counterBucket_balancedCounter[nbPositiveHit]<=nrBucketDesired:
            dict_nbPositiveHit_counterBucket_balancedObtained[nbPositiveHit]+=1
            list_inputBalanced.append(nparray_input[i])
            list_outputBalanced.append(nparray_output[i])
            list_nbPositiveHitBalanced.append(nparray_nbPositiveHit[i])
            list_etaBalanced.append(nparray_eta[i])
            list_layerIDBalanced.append(nparray_layerID[i])
            list_volumeIDBalanced.append(nparray_volumeID[i])
            list_queryIndexBalanced.append(nparray_queryIndex[i])           
        else:
            # do not add it, so that the datasets remained balanced
            #print("Not add nbPositiveHit",nbPositiveHit,"nrBucketDesired",nrBucketDesired,"counter",dict_nbPositiveHit_counterBucket_balancedObtained[nbPositiveHit])
            pass
        # done if
    # done loop over buckets
    
    # now dictionary is filled and we can print the values
    if debug:
        print()
        print("BalancedCounter after. "+TrainOrTest+":")
        for nbPositiveHit in list_nbPositiveHit:
            counterBucket=dict_nbPositiveHit_counterBucket_balancedObtained[nbPositiveHit]
            percentBucket=100*counterBucket/nbBucket
            print("nbPositiveHit=%.0f"%nbPositiveHit,"counterBucket=%.0f"%counterBucket,"percentBucket=%.1f"%percentBucket)

    
    # now dictionary is filled and we can print the values
    if debug or verbose:
        print()
        print("BalancedObtained after. "+TrainOrTest+":")
        for nbPositiveHit in list_nbPositiveHit:
            counterBucket=dict_nbPositiveHit_counterBucket_balancedObtained[nbPositiveHit]
            percentBucket=100*counterBucket/nbBucket
            print("nbPositiveHit=%.0f"%nbPositiveHit,"counterBucket=%.0f"%counterBucket,"percentBucket=%.1f"%percentBucket)

    # convert list to numpy arrays
    nparray_inputBalanced=np.array(list_inputBalanced)
    nparray_outputBalanced=np.array(list_outputBalanced)
    nparray_nbPositiveHitBalanced=np.array(list_nbPositiveHitBalanced)
    nparray_etaBalanced=np.array(list_etaBalanced)
    nparray_layerIDBalanced=np.array(list_layerIDBalanced)
    nparray_volumeIDBalanced=np.array(list_volumeIDBalanced)
    nparray_queryIndexBalanced=np.array(list_queryIndexBalanced)
    #
    p("nparray_inputBalanced",nparray_inputBalanced)
    outputFileNameBalanced=outputFolderNameMerge+"/NN_2_data_InputBalanced_"+TrainOrTest+"_all.npy"
    np.save(outputFileNameBalanced,nparray_inputBalanced)
    #
    p("nparray_outputBalanced",nparray_outputBalanced)
    outputFileNameBalanced=outputFolderNameMerge+"/NN_2_data_OutputBalanced_"+TrainOrTest+"_all.npy"
    np.save(outputFileNameBalanced,nparray_outputBalanced)
    #
    p("nparray_nbPositiveHitBalanced",nparray_nbPositiveHitBalanced)
    outputFileNameBalanced=outputFolderNameMerge+"/NN_2_data_NbPositiveHitBalanced_"+TrainOrTest+"_all.npy"
    np.save(outputFileNameBalanced,nparray_nbPositiveHitBalanced)
    
    p("nparray_etaBalanced",nparray_etaBalanced)
    outputFileNameBalanced=outputFolderNameMerge+"/NN_2_data_EtaBalanced_"+TrainOrTest+"_all.npy"
    np.save(outputFileNameBalanced,nparray_etaBalanced)
    
    p("nparray_layerIDBalanced",nparray_layerIDBalanced)
    outputFileNameBalanced=outputFolderNameMerge+"/NN_2_data_LayerIDBalanced_"+TrainOrTest+"_all.npy"
    np.save(outputFileNameBalanced,nparray_layerIDBalanced)
    
    p("nparray_volumeIDBalanced",nparray_volumeIDBalanced)
    outputFileNameBalanced=outputFolderNameMerge+"/NN_2_data_VolumeIDBalanced_"+TrainOrTest+"_all.npy"
    np.save(outputFileNameBalanced,nparray_volumeIDBalanced)
    
    p("nparray_queryIndexBalanced",nparray_queryIndexBalanced)
    outputFileNameBalanced=outputFolderNameMerge+"/NN_2_data_QueryIndexBalanced_"+TrainOrTest+"_all.npy"
    np.save(outputFileNameBalanced,nparray_queryIndexBalanced)
    
    # make plots
    # we want bins from 0 to 21, as bin 20 is between 20 and 21
    bins=[i-0.5 for i in range(0,22)]
    print("bins",bins)
    # plot not normalized
    plt.hist(nparray_nbPositiveHit,bins=bins,histtype="step",color="red",label="Unbalanced")
    plt.hist(nparray_nbPositiveHitBalanced,bins=bins,histtype="step",color="blue",label="Balanced")
    plt.title("Histogram not normalized for "+TrainOrTest)
    plt.xlabel("Number of positive hits per bucket")
    plt.ylabel("Number of buckets")
    plt.legend(loc=legend_position)
    plt.tight_layout()
    #plt.show()
    plt.savefig(outputFolderNameMerge+"/plot_histo_2_NbBucket_vs_NbPositiveHit_"+TrainOrTest+".png")
    plt.savefig(outputFolderNameMerge+"/plot_histo_2_NbBucket_vs_NbPositiveHit_"+TrainOrTest+".pdf")
    plt.close()
    # plot normalized
    plt.hist(nparray_nbPositiveHit,bins=bins,density=True,histtype="step",color="red",label="Unbalanced")
    plt.hist(nparray_nbPositiveHitBalanced,bins=bins,density=True,histtype="step",color="blue",label="Balanced")
    plt.title("Histogram normalized for "+TrainOrTest)
    plt.xlabel("Number of positive hits per bucket")
    plt.ylabel("Number of buckets")
    plt.legend(loc=legend_position)
    plt.tight_layout()
    #plt.show()
    plt.savefig(outputFolderNameMerge+"/plot_histo_2_NbBucket_vs_NbPositiveHit_normalized_"+TrainOrTest+".png")
    plt.savefig(outputFolderNameMerge+"/plot_histo_2_NbBucket_vs_NbPositiveHit_normalized_"+TrainOrTest+".pdf")
    plt.close()

# done function

In [17]:
def do_balance_mergeOption(mergeOption):
    if debug or verbose:
        print("do_balance_mergeOption(), with mergeOption=",mergeOption)
        
        
    # create output folder name for the merge, for the current number of events to merge
    eventNumberMin=mergeOption[0] # included
    eventNumberMax=mergeOption[1] # excluded
    #
    outputFolderNameMerge=outputFolderName+"_ev_"
    #
    if eventNumberMin<10:
        outputFolderNameMerge+="00"
    elif eventNumberMin<100:
        outputFolderNameMerge+="0"
    outputFolderNameMerge+=str(eventNumberMin)
    #
    outputFolderNameMerge+="_"
    #
    if eventNumberMax<10:
        outputFolderNameMerge+="00"
    elif eventNumberMax<100:
        outputFolderNameMerge+="0"
    outputFolderNameMerge+=str(eventNumberMax)
    #
    if not os.path.exists(outputFolderNameMerge):
        os.makedirs(outputFolderNameMerge)
        
    eventNumber="all"
    
    # Input
    fileNameNNInputTrainAll=outputFolderNameMerge+"/NN_2_data_Input_Train_"+eventNumber+".npy"
    fileNameNNInputTestAll=outputFolderNameMerge+"/NN_2_data_Input_Test_"+eventNumber+".npy"
    nparray_Input_Train_all_Events=np.load(fileNameNNInputTrainAll)
    nparray_Input_Test_all_Events=np.load(fileNameNNInputTestAll)
    if debug:
        print()
        p("nparray_Input_Train_all_Events",nparray_Input_Train_all_Events)
        p("nparray_Input_Test_all_Events",nparray_Input_Test_all_Events) 
    
    # Output
    fileNameNNOutputTrainAll=outputFolderNameMerge+"/NN_2_data_Output_Train_"+eventNumber+".npy"
    fileNameNNOutputTestAll=outputFolderNameMerge+"/NN_2_data_Output_Test_"+eventNumber+".npy"
    nparray_Output_Train_all_Events=np.load(fileNameNNOutputTrainAll)
    nparray_Output_Test_all_Events=np.load(fileNameNNOutputTestAll)
    if debug:
        print()
        p("nparray_Output_Train_all_Events",nparray_Output_Train_all_Events)
        p("nparray_Output_Test_all_Events",nparray_Output_Test_all_Events)  
    
    # NbPositiveHit
    fileNameNNNbPositiveHitTrainAll=outputFolderNameMerge+"/NN_2_data_NbPositiveHit_Train_"+eventNumber+".npy"
    fileNameNNNbPositiveHitTestAll=outputFolderNameMerge+"/NN_2_data_NbPositiveHit_Test_"+eventNumber+".npy"
    nparray_NbPositiveHit_Train_all_Events=np.load(fileNameNNNbPositiveHitTrainAll)
    nparray_NbPositiveHit_Test_all_Events=np.load(fileNameNNNbPositiveHitTestAll)
    if debug:
        print()
        p("nparray_NbPositiveHit_Train_all_Events",nparray_NbPositiveHit_Train_all_Events)
        p("nparray_NbPositiveHit_Test_all_Events",nparray_NbPositiveHit_Test_all_Events)
        
    # Eta
    fileNameNNEtaTrainAll=outputFolderNameMerge+"/NN_2_data_Eta_Train_"+eventNumber+".npy"
    fileNameNNEtaTestAll=outputFolderNameMerge+"/NN_2_data_Eta_Test_"+eventNumber+".npy"
    nparray_Eta_Train_all_Events=np.load(fileNameNNEtaTrainAll)
    nparray_Eta_Test_all_Events=np.load(fileNameNNEtaTestAll)
    if debug:
        print()
        p("nparray_Eta_Train_all_Events",nparray_Eta_Train_all_Events)
        p("nparray_Eta_Test_all_Events",nparray_Eta_Test_all_Events)
        
    # LayerID
    fileNameNNLayerIDTrainAll=outputFolderNameMerge+"/NN_2_data_LayerID_Train_"+eventNumber+".npy"
    fileNameNNLayerIDTestAll=outputFolderNameMerge+"/NN_2_data_LayerID_Test_"+eventNumber+".npy"
    nparray_LayerID_Train_all_Events=np.load(fileNameNNLayerIDTrainAll)
    nparray_LayerID_Test_all_Events=np.load(fileNameNNLayerIDTestAll)
    if debug:
        print()
        p("nparray_LayerID_Train_all_Events",nparray_LayerID_Train_all_Events)
        p("nparray_LayerID_Test_all_Events",nparray_LayerID_Test_all_Events)
        
    # VolumeID
    fileNameNNVolumeIDTrainAll=outputFolderNameMerge+"/NN_2_data_VolumeID_Train_"+eventNumber+".npy"
    fileNameNNVolumeIDTestAll=outputFolderNameMerge+"/NN_2_data_VolumeID_Test_"+eventNumber+".npy"
    nparray_VolumeID_Train_all_Events=np.load(fileNameNNVolumeIDTrainAll)
    nparray_VolumeID_Test_all_Events=np.load(fileNameNNVolumeIDTestAll)
    if debug:
        print()
        p("nparray_VolumeID_Train_all_Events",nparray_VolumeID_Train_all_Events)
        p("nparray_VolumeID_Test_all_Events",nparray_VolumeID_Test_all_Events)
        
    # QueryIndex
    fileNameNNQueryIndexTrainAll=outputFolderNameMerge+"/NN_2_data_QueryIndex_Train_"+eventNumber+".npy"
    fileNameNNQueryIndexTestAll=outputFolderNameMerge+"/NN_2_data_QueryIndex_Test_"+eventNumber+".npy"
    nparray_QueryIndex_Train_all_Events=np.load(fileNameNNQueryIndexTrainAll)
    nparray_QueryIndex_Test_all_Events=np.load(fileNameNNQueryIndexTestAll)
    if debug:
        print()
        p("nparray_QueryIndex_Train_all_Events",nparray_QueryIndex_Train_all_Events)
        p("nparray_QueryIndex_Test_all_Events",nparray_QueryIndex_Test_all_Events)
    
    # for Train and Test
    if True:
        do_balance("Train",nparray_Input_Train_all_Events,nparray_Output_Train_all_Events,nparray_NbPositiveHit_Train_all_Events,nparray_Eta_Train_all_Events,nparray_LayerID_Train_all_Events,nparray_VolumeID_Train_all_Events,nparray_QueryIndex_Train_all_Events,outputFolderNameMerge,doVerifyForHit=True,debug=debug,verbose=True)
        do_balance("Test",nparray_Input_Test_all_Events,nparray_Output_Test_all_Events,nparray_NbPositiveHit_Test_all_Events,nparray_Eta_Test_all_Events,nparray_LayerID_Test_all_Events,nparray_VolumeID_Test_all_Events,nparray_QueryIndex_Test_all_Events,outputFolderNameMerge,doVerifyForHit=True,debug=debug,verbose=True)
    # weights ready
    
# done function

In [18]:
if doBalance:
    for mergeOption in list_mergeOption:
        print()
        print()
        print("******************************")
        print("mergeOption",mergeOption)
        print("******************************")
        do_balance_mergeOption(mergeOption)
        if debug:
            print("Done mergeOption",mergeOption)
    # done for loop over mergeOption
# done if



******************************
mergeOption [0, 30]
******************************
do_balance_mergeOption(), with mergeOption= [0, 30]
Start get_nparray_weight for TrainOrTest= Train
Start nparray_input
[[[ 3.68880e+01]
  [ 1.38507e+00]
  [-1.29750e+03]
  ...
  [ 3.74781e+01]
  [-1.87127e+00]
  [-1.29750e+03]]

 [[ 4.72534e+01]
  [ 3.84431e+00]
  [-1.50250e+03]
  ...
  [ 3.99709e+01]
  [ 5.32750e+00]
  [-1.30250e+03]]

 [[ 7.07738e+01]
  [ 8.25329e+00]
  [-1.30250e+03]
  ...
  [ 4.35503e+01]
  [ 7.16400e+00]
  [-8.22500e+02]]

 ...

 [[-9.78331e+02]
  [ 1.20597e+02]
  [ 2.95550e+03]
  ...
  [-4.93845e+02]
  [ 7.48192e+01]
  [ 1.50450e+03]]

 [[-8.33462e+02]
  [ 1.02791e+02]
  [ 2.54450e+03]
  ...
  [-4.93845e+02]
  [ 7.48192e+01]
  [ 1.50450e+03]]

 [[-9.57845e+02]
  [ 1.09019e+02]
  [ 2.95250e+03]
  ...
  [-2.58630e+02]
  [ 2.93903e+01]
  [ 8.29133e+02]]]
End nparray_input shape (2290382, 60, 1) dtype float32 type <class 'numpy.ndarray'>
Start nparray_output
[[ 1  1  1 ... -1 -1 -1]


Start get_nparray_weight for TrainOrTest= Test
Start nparray_input
[[[ 6.10612e+01]
  [ 5.48271e+00]
  [-1.30250e+03]
  ...
  [ 3.11451e+01]
  [ 3.69277e+00]
  [-6.97500e+02]]

 [[ 5.66778e+01]
  [ 6.69252e+00]
  [-1.30250e+03]
  ...
  [ 5.79303e+01]
  [ 4.73475e+00]
  [-1.30250e+03]]

 [[ 3.15741e+01]
  [ 3.40823e+00]
  [-1.49750e+03]
  ...
  [ 3.07265e+01]
  [-2.34300e+00]
  [-1.49750e+03]]

 ...

 [[-8.80046e+02]
  [ 2.05591e+01]
  [ 2.95250e+03]
  ...
  [-2.59735e+02]
  [-7.12118e+00]
  [ 8.48600e+02]]

 [[-9.59013e+02]
  [ 9.04554e+01]
  [ 2.95250e+03]
  ...
  [-1.14631e+02]
  [ 1.40400e+01]
  [ 3.76046e+02]]

 [[-9.88007e+02]
  [ 1.25375e+02]
  [ 2.95550e+03]
  ...
  [-2.57832e+02]
  [ 3.28188e+01]
  [ 8.18604e+02]]]
End nparray_input shape (947913, 60, 1) dtype float32 type <class 'numpy.ndarray'>
Start nparray_output
[[ 1  1  1 ... -1 -1 -1]
 [ 1  1  1 ... -1 -1 -1]
 [-1 -1 -1 ...  1 -1 -1]
 ...
 [ 1  1  1 ... -1 -1 -1]
 [-1 -1 -1 ... -1 -1 -1]
 [ 1  1  1 ... -1 -1  1]]
End npa

Start nparray_queryIndexBalanced
[[1]
 [3]
 [1]
 ...
 [0]
 [0]
 [1]]
End nparray_queryIndexBalanced shape (609574, 1) dtype uint8 type <class 'numpy.ndarray'>
bins [-0.5, 0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5, 10.5, 11.5, 12.5, 13.5, 14.5, 15.5, 16.5, 17.5, 18.5, 19.5, 20.5]


In [20]:
def do_process_merged(mergeOption):
    
    # create output folder name for the merge, for the current number of events to merge
    eventNumberMin=mergeOption[0] # included
    eventNumberMax=mergeOption[1] # excluded
    #
    outputFolderNameMerge=outputFolderName+"_ev_"
    #
    if eventNumberMin<10:
        outputFolderNameMerge+="00"
    elif eventNumberMin<100:
        outputFolderNameMerge+="0"
    outputFolderNameMerge+=str(eventNumberMin)
    #
    outputFolderNameMerge+="_"
    #
    if eventNumberMax<10:
        outputFolderNameMerge+="00"
    elif eventNumberMax<100:
        outputFolderNameMerge+="0"
    outputFolderNameMerge+=str(eventNumberMax)
    #
    if not os.path.exists(outputFolderNameMerge):
        os.makedirs(outputFolderNameMerge)
    
    
    # create the numpy array with all negative with which we replace those that need to
    nparray_output_negative_one_bucket=np.array([-1 for i in range(bucketSize)]).astype(np.int8)
    
    # we change outputC for both Train and Test
    list_TrainOrTest=["Train","Test"]
    
    eventNumber="all"
    
    # for loop over TrainOrTest
    for TrainOrTest in list_TrainOrTest:
        if debug or verbose:
            print("TrainOrTest",TrainOrTest)
        # open the files needed for this event: Output and NbPositiveHit
        fileNameNNOutputAll=outputFolderNameMerge+"/NN_2_data_OutputBalanced_"+TrainOrTest+"_"+eventNumber+".npy"
        nparray_output_all=np.load(fileNameNNOutputAll)
        fileNameNNNbPositiveHitAll=outputFolderNameMerge+"/NN_2_data_NbPositiveHitBalanced_"+TrainOrTest+"_"+eventNumber+".npy"
        nparray_nbPositiveHit_all=np.load(fileNameNNNbPositiveHitAll)
        # 
        dict_minNbPositiveHit_nparray_output_all={}
        # for loop over each val of min number of positive fits
        for minNbPositiveHit in list_minNbPositiveHit:
            if debug:
                print("i",i,"eventNumber",eventNumber,"minNbPositiveHit",minNbPositiveHit)
            # copy the current output in a completely new version with deep copy
            # as we will want to replace only the values that need changed for less than needed positive hits
            dict_minNbPositiveHit_nparray_output_all[minNbPositiveHit]=copy.deepcopy(nparray_output_all)
        # done for loop over minNbPositiveHit
        #
        # loop over each row (bucket) in the output
        for i in range(len(nparray_output_all)):
            nbPositiveHit=nparray_nbPositiveHit_all[i][0]
            if debug:
                print("i=%.0f, nbPositiveHit=%.0f"%(i,nbPositiveHit))
            # to speed up, change only wheen needed
            for minNbPositiveHit in list_minNbPositiveHit:
                if nbPositiveHit<minNbPositiveHit:
                    # replace this one line with the one with all negative
                    dict_minNbPositiveHit_nparray_output_all[minNbPositiveHit][i]=nparray_output_negative_one_bucket
                if debug:
                    print("i=%.0f, minNbPositiveHit=%.0f, bucket"%(i,minNbPositiveHit),dict_minNbPositiveHit_nparray_output_all[minNbPositiveHit][j])
            # done for loop over minNbPositiveHit  
        # done for loop over buckets
        # write the new processed output to file
        for minNbPositiveHit in list_minNbPositiveHit:
            minString="Min"
            if minNbPositiveHit<9:
                minString+="0"
            minString+=str(minNbPositiveHit)
            fileNameNNOutputAll=outputFolderNameMerge+"/NN_2_data_OutputBalanced"+minString+"_"+TrainOrTest+"_"+eventNumber+".npy"
            np.save(fileNameNNOutputAll,dict_minNbPositiveHit_nparray_output_all[minNbPositiveHit])
        # done for loop over minNbPositiveHit  
    # done for loop over events
# done function
if debug or verbose:
    print("Done do_create_output_based_on_min_nb_of_positive_hits(list_eventNumber")

Done do_create_output_based_on_min_nb_of_positive_hits(list_eventNumber


In [21]:
if doProcessMerged:
    for mergeOption in list_mergeOption:
        print()
        print()
        print("******************************")
        print("mergeOption",mergeOption)
        print("******************************")
        do_process_merged(mergeOption)
        if debug:
            print("Done mergeOption",mergeOption)
    # done for loop over mergeOption
# done if



******************************
mergeOption [0, 30]
******************************
TrainOrTest Train
TrainOrTest Test


In [ ]:
print("Done all!")